In [36]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact
import pandas as pd
import psycopg2 as pg
import sqlalchemy
from sqlalchemy import create_engine
import datetime

## Conexão com o banco de dados

In [37]:
cnx = 'postgresql://postgres:postgres@db:5432/acessiview'
sqlalchemy.create_engine(cnx)

Engine(postgresql://postgres:***@db:5432/acessiview)

In [38]:
con = pg.connect(host='db', dbname= 'acessiview', user='postgres', password= 'postgres')

## Definição da Aplicação

Acessiview é uma aplicação simples, onde pessoas com deficiência podem encontrar estabelecimentos que tenham tags específicas que tratem algum tipo de acessibilidade dentro de suas lojas. Por exemplo, uma pessoa que necessita se locomovem com cadeira de rodas e quer ir em um estabelecimento específico, pode verificar se ele existe no sistema e, caso queira, ver se o estabelecimento tem uma tag para cadeirantes como "Rampas", ou no caso de um restaurante uma tag como "banheiros acessíveis para cadeirantes", essas tags são criadas no sistema, ou sugeridas para criação e as pessoas que forem nos estabelecimentos podem verificar a veracidade das tags que foram adicionadas, dando um like ou comentário como avaliação, logo, outras pessoas que forem usar o aplicativo para encontrar estabelecimentos podem checar a confiabilidade das tags de um estabelecimento pelas avaliações feitas ao mesmo.

## CRUDS


#### USUÁRIOS

#### READ


In [39]:
flag=''

input_search = widgets.Text(
    value='',
    placeholder='Digite um nome',
    disabled=False
)

button_search = widgets.Button(description="Search", value='')

def on_button_clicked_query(b):
    query = ''
    input_text = input_search.value
    
    try:
        if(input_search.value == ''):   
            query = f"select * from usuario"
        else:
            query = f"select * from usuario where nome='{input_text}'"
            
        df = pd.read_sql_query(query, cnx)
        display(input_search, button_search, df)
        
    except:        
        display(input_search, button_search)    
        display("Não foi possível realizar a consulta.")
        
button_search.on_click(on_button_clicked_query)

output = widgets.Output()

with output:
    output.clear_output()
    display(input_search, button_search)

output

        
        

Output()

#### ACTIONS

In [45]:
#CAMPOS DO USUÁRIO
cod = widgets.IntText(
    value = 0,
    description='Cod: ',
    disabled=False
)

nome = widgets.Text(
    value='',
    placeholder='Digite seu nome',
    description='Nome: ',
    disabled=False
)

qtd_comentarios = widgets.IntText(
    value = 0,
    description='Qtd. Comentários: ',
    disabled=True
)

dt_criacao = datetime.date.today().strftime("%d-%m-%Y")

foto = widgets.Text(
    value='URL',
    description='Foto: ',
    disabled=False
)

qtd_avaliacoes = widgets.IntText(
    value = 0,
    description='Qtd. Avaliações: ',
    disabled=True
)

data_nasc = widgets.DatePicker(
    description='Data de Nascimento:',
    disabled=False
)

button_insert = widgets.Button(description="Inserir", value='')
button_update = widgets.Button(description="Update", value='')
button_delete = widgets.Button(description="Delete", value='')

def on_button_clicked_insert(b):
    try:
        cursor = con.cursor()
        convert_date = data_nasc.value
        cursor.execute("insert into public.usuario(nome, qtd_comentarios, dt_criacao, foto, qtd_avaliacoes, dt_nasc) values (%s, %s, %s, %s, %s, %s)", (nome.value, qtd_comentarios.value, dt_criacao, foto.value, qtd_avaliacoes.value, convert_date.strftime("%d-%m-%Y")))
                              
        cursor.query
        con.commit()
        df = pd.read_sql("select * from usuario", cnx)
        display(cod, nome, qtd_comentarios, foto, qtd_avaliacoes, data_nasc, df)
        display(button_insert, button_update, button_delete)

    except:
        cursor.execute("ROLLBACK")   
        display(cod, nome, qtd_comentarios, foto, qtd_avaliacoes, data_nasc)
        display(button_insert, button_update, button_delete)
        display("Não foi possível realizar a inserção. Verifique restrições.")
        
    finally:
        cursor.close()
        
        
def on_button_clicked_update(b):
    try:
        cursor= con.cursor()
        cursor.execute("update public.usuario (nome, foto, dt_nasc) set (nome=%s, foto=%s, dt_nasc=%s) where cod=%s", (nome.value, foto.value, convert_date.strftime("%d-%m-%Y"), cod.value))
        
        cursor.query
        con.commit()
        
        df = pd.read_sql("select * from usuario", cnx)
        display(cod, nome, qtd_comentarios, foto, qtd_avaliacoes, data_nasc, df)
        display(button_insert, button_update, button_delete)
    except:
        cursor.execute("ROLLBACK")   
        display(cod, nome, qtd_comentarios, foto, qtd_avaliacoes, data_nasc)
        display(button_insert, button_update, button_delete)
        display("Não foi possível realizar a atualização. Verifique restrições.")
            
    finally:
        cursor.close()
        
        
def on_button_clicked_delete(b):
    try:
        print(nome.value)
        cursor= con.cursor()
        cursor.execute("delete from public.usuario where nome=%s", (nome.value,))
        rows_deleted = cursor.rowcount
        con.commit()
        df = pd.read_sql("select * from usuario", cnx)
        display(cod, nome, qtd_comentarios, foto, qtd_avaliacoes, data_nasc, df)
        display(button_insert, button_update, button_delete)
    except:
        cursor.execute("ROLLBACK")   
        display(cod, nome, qtd_comentarios, foto, qtd_avaliacoes, data_nasc)
        display(button_insert, button_update, button_delete)
        display("Não foi possível realizar a deleção. Verifique restrições.")
            
    finally:
        cursor.close()
        
button_insert.on_click(on_button_clicked_insert)
button_update.on_click(on_button_clicked_update)
button_delete.on_click(on_button_clicked_delete)

output = widgets.Output()

with output:
    output.clear_output()
    display(cod, nome, qtd_comentarios, foto, qtd_avaliacoes, data_nasc)
    display(button_insert, button_update, button_delete)
    


output

Output()